In [1]:
_RIOT_API_KEY='RGAPI-f133c0d1-25f4-4ab2-ad9e-7d96aa3b73f5'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import constants
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from itertools import compress

lol_watcher = LolWatcher(_RIOT_API_KEY)

In [2]:
#match = lol_watcher.match.by_id('na1', 'NA1_3891774034')
#match = lol_watcher.match.by_id('na1', 'NA1_4720874733')
engine = help.create_mysql_engine()

# Plan for New Role Classification
1. Create new DB table 'lol_match_roles' with: matchId, participantId, Role, validRoles
2. Re-Create function for Jungle Flag
2. Re-Create function for Support Flag
3. Re-Create function for Lane By Posx/Posy
4. Create Mother Function 'determine_roles' To Validate positions for a match. validRoles = True if 1 player in each role, False otherwise. Upload to new 
5. Probably create a 'lol_analysis_view' which pulls needed info from 'lol_match_details', 'lol_match_timeline', and 'lol_match_roles' which an analysis script can pull from.


In [4]:
rc.determine_roles(engine)

0 matches with more than 2 smites:
0 matches with more than 2 champs with support items:
2 match roles uploaded, including: 
['NA1_4747898509' 'NA1_4747868600'] match roles uploaded, including: 


In [ ]:
def fillin_role(df):
    l = list(df['role'])

    role_dict = {
            'TOP':'TOP' in l,
            'JUNGLE' : 'JUNGLE' in l,
            'MIDDLE' : 'MIDDLE' in l,
            'BOTTOM' : 'BOTTOM' in l,
            'SUPPORT' : 'SUPPORT' in l
        }
        
    # If we're only missing one role, return that. If len <5 then there's a duplicate
    if len(Counter(l)) == 5 and role_dict['TOP'] + role_dict['JUNGLE'] + role_dict['MIDDLE'] + role_dict['BOTTOM'] + role_dict['SUPPORT']==4:
        mask = [not elem for elem in list(role_dict.values())]
        leftover = list(compress(list(role_dict.keys()), mask))[0]
        return leftover
    else:
        return 'Invalid'
        
def determine_roles(db_engine):

    to_classify = find_to_classify(db_engine)
    jung = classify_jungler(to_classify, db_engine)
    supp = classify_support(to_classify, db_engine)
    pos = classify_roles_by_position(to_classify, db_engine)

    df = jung.merge(supp[['matchid','participantid','support_flag']],
                        how='inner', left_on = ['matchid','participantid'],
                        right_on = ['matchid','participantid']).merge(pos[['matchid','participantid','pos_lane']],
                                                                        how='inner', left_on = ['matchid','participantid'],
                                                                        right_on = ['matchid','participantid'])
    df['role'] = np.vectorize(define_role)(df['jungle_flag'], df['support_flag'], df['pos_lane'])

    # For the matches with a None, see if the None is just the odd man out and there are 9 other unique Roles
    for match_id, teamName, index in zip(list(df[df['role']=='None'].matchid),
                                        list(df[df['role']=='None'].teamName),
                                        list(df[df['role']=='None'].index)):
        x = df[(df['matchid']==match_id) & (df['teamName']==teamName)]
        df.loc[index, 'role'] = fillin_role(x)

    invalid_matches = df.groupby('matchid')['role'].value_counts().reset_index()
    unique_invalid_matches = invalid_matches[invalid_matches['count']!=2].matchid.unique()
    df['validRole'] = [False if df.loc[i,'matchid'] in unique_invalid_matches else True for i in range(len(df)) ]
    df[['matchid','participantid','role','validRole']].to_sql(con=db_engine, name='lol_match_roles', if_exists='append', index = False)

In [7]:
to_classify = rc.find_to_classify(engine)
to_classify = ['NA1_4691557046', 'NA1_4691557046']
#print(to_classify)
jung = rc.classify_jungler(to_classify, engine)
supp = rc.classify_support(to_classify, engine)
pos = rc.classify_roles_by_position(to_classify, engine)

0 matches with more than 2 smites:
0 matches with more than 2 champs with support items:


In [16]:
query=f"""      SELECT 
                            matchid,
                            participantid,
                            minute,
                            posX,
                            posY
                        FROM riot_api.lol_match_timeline
                        where matchId ='NA1_4687095094' and participantid=9 """
df = pd.read_sql(query, con=engine)

df['pos_lane'] = np.vectorize(rc.classify_position)(df['posX'], df['posY'])
df

,matchid,participantid,minute,posX,posY,pos_lane
0,NA1_4687095094,9,0,14404,14201,None
1,NA1_4687095094,9,1,8293,3616,BOTTOM
2,NA1_4687095094,9,2,11527,3728,BOTTOM
3,NA1_4687095094,9,3,11620,2007,BOTTOM
4,NA1_4687095094,9,4,13741,8844,BOTTOM
5,NA1_4687095094,9,5,12234,1980,BOTTOM
6,NA1_4687095094,9,6,13316,5678,BOTTOM
7,NA1_4687095094,9,7,13243,3608,BOTTOM
8,NA1_4687095094,9,8,13148,4420,BOTTOM
9,NA1_4687095094,9,9,13705,7771,BOTTOM


In [8]:
df = jung.merge(supp[['matchid','participantid','support_flag']],
 how='inner', left_on = ['matchid','participantid'],
  right_on = ['matchid','participantid']).merge(pos[['matchid','participantid','pos_lane']],
                                                how='inner', left_on = ['matchid','participantid'],
                                                right_on = ['matchid','participantid'])
df['role'] = np.vectorize(rc.define_role)(df['jungle_flag'], df['support_flag'], df['pos_lane'])
df

,matchid,teamName,participantid,jungle_flag,support_flag,pos_lane,role
0,NA1_4691557046,Blue,1,False,False,TOP,TOP
1,NA1_4691557046,Blue,2,True,False,TOP,JUNGLE
2,NA1_4691557046,Blue,3,False,False,MIDDLE,MIDDLE
3,NA1_4691557046,Blue,4,False,False,BOTTOM,BOTTOM
4,NA1_4691557046,Blue,5,False,True,BOTTOM,SUPPORT
5,NA1_4691557046,Red,6,False,False,TOP,TOP
6,NA1_4691557046,Red,7,True,False,None,JUNGLE
7,NA1_4691557046,Red,8,False,False,MIDDLE,MIDDLE
8,NA1_4691557046,Red,9,False,False,None,None
9,NA1_4691557046,Red,10,False,False,BOTTOM,BOTTOM


In [9]:
df

,matchid,teamName,participantid,jungle_flag,support_flag,pos_lane,role
0,NA1_4691557046,Blue,1,False,False,TOP,TOP
1,NA1_4691557046,Blue,2,True,False,TOP,JUNGLE
2,NA1_4691557046,Blue,3,False,False,MIDDLE,MIDDLE
3,NA1_4691557046,Blue,4,False,False,BOTTOM,BOTTOM
4,NA1_4691557046,Blue,5,False,True,BOTTOM,SUPPORT
5,NA1_4691557046,Red,6,False,False,TOP,TOP
6,NA1_4691557046,Red,7,True,False,None,JUNGLE
7,NA1_4691557046,Red,8,False,False,MIDDLE,MIDDLE
8,NA1_4691557046,Red,9,False,False,None,None
9,NA1_4691557046,Red,10,False,False,BOTTOM,BOTTOM


In [178]:
# For the matches with a None, see if the None is just the odd man out and there are 9 other unique Roles
for match_id, teamName, index in zip(list(df[df['role']=='None'].matchid),
                                     list(df[df['role']=='None'].teamName),
                                     list(df[df['role']=='None'].index)):
    x = df[(df['matchid']==match_id) & (df['teamName']==teamName)]
    df.loc[index, 'role'] = fillin_role(x)

invalid_matches = df.groupby('matchid')['role'].value_counts().reset_index()
unique_invalid_matches = invalid_matches[invalid_matches['count']!=2].matchid.unique()
df['validRole'] = [False if df.loc[i,'matchid'] in unique_invalid_matches else True for i in range(len(df)) ]


32790

False

,matchid,teamName,participantid,jungle_flag,support_flag,pos_lane,role,validRole
0,NA1_4720874733,Blue,1,False,False,TOP,TOP,True
1,NA1_4720874733,Blue,2,True,False,None,JUNGLE,True
2,NA1_4720874733,Blue,3,False,False,MIDDLE,MIDDLE,True
3,NA1_4720874733,Blue,4,False,False,BOTTOM,BOTTOM,True
4,NA1_4720874733,Blue,5,False,True,BOTTOM,SUPPORT,True
...,...,...,...,...,...,...,...,...
32785,NA1_3986338932,Red,6,False,False,TOP,TOP,True
32786,NA1_3986338932,Red,7,True,False,None,JUNGLE,True
32787,NA1_3986338932,Red,8,False,False,MIDDLE,MIDDLE,True
32788,NA1_3986338932,Red,9,False,False,BOTTOM,BOTTOM,True


,matchid
0,NA1_3699835860
1,NA1_3700655024
2,NA1_3700733373
3,NA1_3701839460
4,NA1_3701924445
...,...
282,NA1_4685587569
283,NA1_4691977053
284,NA1_4713234695
285,NA1_4713633775


In [103]:
def fillin_role(df):
    l = list(df['role'])

    role_dict = {
            'TOP':'TOP' in l,
            'JUNGLE' : 'JUNGLE' in l,
            'MIDDLE' : 'MIDDLE' in l,
            'BOTTOM' : 'BOTTOM' in l,
            'SUPPORT' : 'SUPPORT' in l
        }
        
    # If we're only missing one role, return that. If len <5 then there's a duplicate
    if len(Counter(l)) == 5 and role_dict['TOP'] + role_dict['JUNGLE'] + role_dict['MIDDLE'] + role_dict['BOTTOM'] + role_dict['SUPPORT']==4:
        mask = [not elem for elem in list(role_dict.values())]
        leftover = list(compress(list(role_dict.keys()), mask))[0]
        return leftover
    else:
        return 'Invalid'

# Classify Lane by Coordinates

In [70]:
minutes_to_pull = [3,4,5,6,7,8]
query=f"""  SELECT 
                    matchid,
                    participantid,
                    minute,
                    posX,
                    posY
                FROM riot_api.lol_match_timeline
                where matchId in {tuple(to_classify[:2])} and minute in {tuple(minutes_to_pull)} """
df = pd.read_sql(query, con=engine)
df

,matchid,participantid,minute,posX,posY
0,NA1_4720874733,1,3,2549,12901
1,NA1_4720874733,2,3,3847,6202
2,NA1_4720874733,3,3,7411,6300
3,NA1_4720874733,4,3,10921,1386
4,NA1_4720874733,5,3,10941,1447
...,...,...,...,...,...
115,NA1_4716080477,6,8,13093,2553
116,NA1_4716080477,7,8,11934,3655
117,NA1_4716080477,8,8,2046,12389
118,NA1_4716080477,9,8,7896,7339


In [73]:
df[df['participantid']=='1']

,matchid,participantid,minute,posX,posY
0,NA1_4720874733,1,3,2549,12901
10,NA1_4720874733,1,4,1234,9087
20,NA1_4720874733,1,5,547,820
30,NA1_4720874733,1,6,1558,11719
40,NA1_4720874733,1,7,2777,12473
50,NA1_4720874733,1,8,669,3076
60,NA1_4716080477,1,3,11400,1635
70,NA1_4716080477,1,4,10965,1396
80,NA1_4716080477,1,5,10279,1398
90,NA1_4716080477,1,6,11458,1338


In [100]:
top_area=Polygon([(0,16000),(0,6000),(3950,6000),(4800,11700),(11000,12500),(11000,16000)])
mid_area=Polygon([(6000,3700),(9500,5000),(11500,7500),(12500,10500),(10500,12000),(7000,10500),(5000,9000),(4200,6000)])
bot_area=Polygon([(16000,0),(6000,0),(6000,3500),(12000,4000),(13000,10000),(16000,10000)])

def classify_position(x,y):
    pt = Point(x,y)
    if bot_area.contains(pt):
        return "BOTTOM"
    elif mid_area.contains(pt):
        return "MIDDLE"
    elif top_area.contains(pt):
        return "TOP"
    else:
        return "None"

In [101]:
to_classify = rc.find_to_classify(engine)
tuple(to_classify[:2])

('NA1_4720874733', 'NA1_4716080477')

In [108]:
def classify_roles_by_position(match_list_to_classify, db_engine, minutes_to_pull = [3,4,5,6,7,8]):
    query=f"""      SELECT 
                        matchid,
                        participantid,
                        minute,
                        posX,
                        posY
                    FROM riot_api.lol_match_timeline
                    where matchId in {tuple(match_list_to_classify)} and minute in {tuple(minutes_to_pull)} """
    df = pd.read_sql(query, con=db_engine)

    df['posLane'] = np.vectorize(classify_position)(df['posX'], df['posY'])
    position_flag_df=df.groupby(['matchid','participantid'])['posLane'].agg(lambda x:x.value_counts().index[0])
    return position_flag_df.reset_index()
classify_roles_by_position(to_classify, engine)

,matchid,participantid,posLane
0,NA1_3699835860,1,TOP
1,NA1_3699835860,10,BOTTOM
2,NA1_3699835860,2,BOTTOM
3,NA1_3699835860,3,MIDDLE
4,NA1_3699835860,4,BOTTOM
...,...,...,...
32785,NA1_4745789794,5,BOTTOM
32786,NA1_4745789794,6,TOP
32787,NA1_4745789794,7,MIDDLE
32788,NA1_4745789794,8,MIDDLE


In [107]:
query=f"""      SELECT 
                    matchid,
                    participantid,
                    minute,
                    posX,
                    posY
                FROM riot_api.lol_match_timeline
                where matchId in {tuple(to_classify)} and minute in {tuple(minutes_to_pull)} """
df = pd.read_sql(query, con=engine)

df['posLane'] = np.vectorize(classify_position)(df['posX'], df['posY'])
position_flag_df=df.groupby(['matchid','participantid'])['posLane'].agg(lambda x:x.value_counts().index[0])
position_flag_df.reset_index()

,matchid,participantid,posLane
0,NA1_3699835860,1,TOP
1,NA1_3699835860,10,BOTTOM
2,NA1_3699835860,2,BOTTOM
3,NA1_3699835860,3,MIDDLE
4,NA1_3699835860,4,BOTTOM
...,...,...,...
32785,NA1_4745789794,5,BOTTOM
32786,NA1_4745789794,6,TOP
32787,NA1_4745789794,7,MIDDLE
32788,NA1_4745789794,8,MIDDLE


In [ ]:

def lane_by_position(df,mins=[3,4,5,6,7,8,9,10],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0]):

    df_mins['posLane'] = np.vectorize(classify_position)(df_mins['posX'], df_mins['posY'])
    #make df smaller for group by
    df_mins2=df_mins[['summonerName','gameId','posLane']].copy()
    df_mins2 = df_mins2[df_mins2['posLane']!="None"]
    #take the mode of where the champion's calculated lane should be'
    position_flag_df=df_mins2.groupby(['summonerName','gameId']).agg(lambda x:x.value_counts().index[0])
    
    df_merge=df.merge(position_flag_df,on=['summonerName','gameId'])
    #want to return OG dataframe, but with another column of the new mapping
    return df_merge